
# Notebook de Ingestão de Dados - Camada Bronze

Este notebook tem como objetivo realizar a ingestão de dados para a camada bronze, responsável por armazenar os dados brutos provenientes das fontes originais. Os dados adicionados neste processo são provenientes do portal de transparência do governo federal.

Serão armazenados dois conjuntos de dados:
- As emendas parlamentares, referentes aos anos de 2023 até 2025, que foram adicionadas diretamente pelo carregamento de um arquivo CSV.
- Os parlamentares em exercício de 2023 a 2026, obtidos via requisições e ingeridos neste notebook.

# Configuração do Ambiente

## Carregando bibliotecas

In [0]:
import requests
from PIL import Image
from IPython.display import display

## Criando Schema Bronze

In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS bronze")

# Criando tabelas com os dados

## Parlamentares

In [0]:
# Url para requisição de deputados em exercício
url = "https://dadosabertos.camara.leg.br/api/v2/deputados"

# Pegando todos os deputados em exercício (mandato atual)
params = {
    "ordem": "ASC",
    "ordenarPor": "nome",
    "pagina": 1,
    "itens": 100
}

todos = []
while True:
    resp = requests.get(url, params=params)
    resp.raise_for_status()

    dados = resp.json()["dados"]
    todos.extend(dados)

    # Verifica se acabou a paginação
    if "links" in resp.json():
        next_link = [lnk for lnk in resp.json()["links"] if lnk["rel"] == "next"]
        if next_link:
            params["pagina"] += 1
        else:
            break
    else:
        break

# Converter JSON para Spark DataFrame
df = spark.createDataFrame(todos)

# Gravar no bronze (Delta)
df.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("bronze.parlamentares")


In [0]:
df.display()


> Tabela criada com sucesso contendo 513 linhas, correspondente ao número de deputados federais no Brasil.  
> Para esses deputados, foram ingeridas as seguintes colunas:
> - email
> - id
> - idLegislatura (57 é o mandato atual do Brasil)
> - nome
> - siglaPartido
> - siglaUF
> - uri
> - uriPartido
> - uriFoto

## Emendas


As emendas parlamentares foram adicionadas diretamente no catálogo utilizando um arquivo CSV.

In [0]:
%sql
select *
from workspace.bronze.emendas
limit 5

In [0]:
%sql
select count(*)
from workspace.bronze.emendas


> A carga das emendas parlamentares foi realizada corretamente, contendo 18350 linhas e as seguintes colunas:
> - Ano: Foram carregados os anos 2023, 2024 e 2025
> - Tipo de Emenda
> - Autor da emenda
> - Número da emenda
> - Possui Apoiador/Solicitante?
> - Localidade do gasto (Regionalização)
> - Função
> - Subfunção
> - Programa Orçamentário
> - Ação Orçamentária
> - Plano Orçamentário
> - Código da emenda
> - Valor empenhado
> - Valor liquidado
> - Valor pago
> - Valor Restos a Pagar Inscritos
> - Valor Restos a Pagar Cancelados
> - Valor Restos a Pagar Pagos

# Evidência de documentação do catálogo

## Parlamentares

In [0]:
# Printa imagem na tela
caminho_imagem_parlamentares = "/Workspace/Users/vmaquino77@gmail.com/pucrio-mvp-engenharia-dados/fotos/bronze_parlamentares_databricks.png"
imagem_parlamentares = Image.open(caminho_imagem_parlamentares)
display(imagem_parlamentares)

## Emendas 

In [0]:
# Printa imagem na tela
caminho_imagem_emendas = "/Workspace/Users/vmaquino77@gmail.com/pucrio-mvp-engenharia-dados/fotos/bronze_emendas_databricks.png"
imagem_emendas = Image.open(caminho_imagem_emendas)
display(imagem_emendas)